# Data Collection

This is our actual file for data collection. 

In [1]:
from googleapiclient.discovery import build
import pandas as pd

In [2]:
api_key = "" #REMEMBER TO REMOVE BEFORE UPLOADING
api_service_name = "youtube"
api_version = "v3"
youtube = build(
    api_service_name, api_version, developerKey=api_key
)

### Get channel Data

In [5]:
def get_channel_info(userid):
    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id=userid
    )
    response = request.execute()
    item = response['items'][0]
    # Your solution
    return {
        'channelName': item['snippet']['title'],
        'channelStartDate': item['snippet']['publishedAt'],
        'subscribers': item['statistics']['subscriberCount'],
        'viewCount': item['statistics']['viewCount'],
        'videoCount': item['statistics']['videoCount'],
        'uploadsPlaylist': item['contentDetails']['relatedPlaylists']['uploads']
    }
def get_video_data(video_ids):
    video_data = []
    for i in range (0, len(video_ids), 50): # performs requests in batches to avoid rate-limiting
        request = youtube.videos().list(
            part='snippet,contentDetails,statistics',
            id=','.join(video_ids[i:i+50])
        )
        response = request.execute() #record response

        for item in response['items']:
            relevant_stats = {
                'snippet': ['title', 'description', 'tags', 'publishedAt'], 
                'statistics': ['viewCount', 'likeCount', 'commentCount'], 
                'contentDetails': ['duration', 'definition', 'caption']
            } #collects information that we care about... check documentation to choose information

            video_info = {}
            video_info['video_id'] = item['id']

            for cat in relevant_stats.keys():
                for stat in relevant_stats[cat]:
                    try:
                        video_info[stat] = item[cat][stat]
                    except:
                        video_info[stat] = None

            video_data.append(video_info)
    video_data
    return pd.DataFrame(video_data)

# Define any helper functions here
def get_video_ids(playlistID):
    request = youtube.playlistItems().list(
        part='snippet,contentDetails',
        playlistId = playlistID,
        maxResults=50
    )

    response = request.execute()

    video_ids = []
    video_ids.extend([item['contentDetails']['videoId'] for item in response['items']])

    next_page = response.get('nextPageToken')

    while next_page is not None: 
        request = youtube.playlistItems().list(
            part='snippet,contentDetails',
            playlistId = playlistID,
            maxResults = 50,
            pageToken = next_page 
        )
        response = request.execute()

        video_ids.extend([item['contentDetails']['videoId'] for item in response['items']])
        next_page = response.get('nextPageToken')

    return video_ids

def get_channel_data(userid):
    channel_info = get_channel_info(userid)
    shorts_playlistid = "UUSH" + userid[2:]

    video_ids = get_video_ids(shorts_playlistid)
    upload_data = get_video_data(video_ids)

    return channel_info, upload_data

In [9]:
# Misinformative channel ids
mis1_id = "UC3w193M5tYPJqF0Hi-7U-2g" #dr berg
mis2_id = "UC5apkKkeZQXRSDbqSalG8CQ" #shawn baker
mis3_id = "UCgBg0LcHfnJDPmFTTf677Pw" #paul saladino

In [10]:
# Informative channel ids
inf1_id = "UCcffZfMDLakH-hs89uSKxQg" #nutrition by kylie
inf2_id = "UCKLz-9xkpPNjK26PqbjHn7Q" #abbey sharp

In [13]:
mis1_info, mis1_uploads = get_channel_data(mis1_id)
mis2_info, mis2_uploads = get_channel_data(mis2_id)
mis3_info, mis3_uploads = get_channel_data(mis3_id)

In [14]:
inf1_info, inf1_uploads = get_channel_data(inf1_id)
inf2_info, inf2_uploads = get_channel_data(inf2_id)

### Get Comments